### Pong

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus and then use the mouse to control the pedal.

* pong sound is by [freesound](https://freesound.org/people/NoiseCollector/sounds/4359/).
* Commodore 64 font is by [KreativeKorp](https://www.kreativekorp.com/software/fonts/c64.shtml).

In [1]:
import pyglet
import math
import time
import sys
import os

import PIL.Image

import numpy as np

In [2]:
p0 = os.path.abspath('.')
p1 = os.path.abspath(os.path.join(p0, '..'))

sys.path.insert(0, p1)

In [3]:
import jupylet

from jupylet import *

In [4]:
import pyglet.window.key as key

In [5]:
app = App(mode='jupyter')

In [6]:
window = app.window

In [7]:
WIDTH = app.width
HEIGHT = app.height

In [8]:
background = '#3e32a2'
foreground = '#7c71da'

In [9]:
app.set_window_color(foreground)

In [10]:
a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((HEIGHT * 9 // 10, WIDTH * 9 // 10, 3)) * jupylet.color2rgb(background)[:3]

In [11]:
ball = Sprite(a0, y=HEIGHT/2, x=WIDTH/2)

padl = Sprite(a1, y=HEIGHT/2, x=48)
padr = Sprite(a1, y=HEIGHT/2, x=WIDTH-48)

field = Sprite(a2, y=HEIGHT/2, x=WIDTH/2) 

In [12]:
sound = pyglet.media.load('sounds/pong-blip.wav', streaming=False)

In [13]:
pyglet.font.add_file('fonts/PetMe64.ttf')

In [14]:
scorel = Label(
    '0', font_name='Pet Me 64', font_size=42, color=foreground, 
    x=64, y=HEIGHT/2, anchor_y='center', anchor_x='left'
)

scorer = Label(
    '0', font_name='Pet Me 64', font_size=42, color=foreground, 
    x=WIDTH-64, y=HEIGHT/2, anchor_y='center', anchor_x='right'
)

In [15]:
sl = 0
sr = 0

In [16]:
@app.event
def on_draw():
    
    window.clear()
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()

In [17]:
pvy = 0
yyy = HEIGHT/2

In [18]:
@app.event
def on_mouse_motion(x, y, dx, dy):
    
    global yyy
    yyy = y
    
    
@app.run_me_again_and_again(1/120)
def update_pads(dt):
    
    global pvy
    
    avy = 200 * (yyy - padr.y)
    pvy = pvy * 0.9 + (avy * dt)
    
    padr.y += pvy * dt
    padl.y += pvy * dt
    
    padr.clip_position(WIDTH, HEIGHT)
    padl.clip_position(WIDTH, HEIGHT)

In [19]:
bvx = 192
bvy = 192

In [20]:
@app.run_me_again_and_again(1/120)
def update_ball(dt):
    
    global bvx, bvy, sl, sr

    ball.x += bvx * dt
    ball.y += bvy * dt
    
    if ball.top >= HEIGHT:
        app.play_once(sound)
        ball.y -= ball.top - HEIGHT
        bvy = -bvy
        
    if ball.bottom <= 0:
        app.play_once(sound)
        ball.y -= ball.bottom
        bvy = -bvy
        
    if ball.right >= WIDTH:
        app.play_once(sound)
        ball.x -= ball.right - WIDTH
        bvx = -bvx
        bvy = 192 * np.sign(bvy)
        
        sl += 1
        scorel.text = str(sl)
        
    if ball.left <= 0:
        app.play_once(sound)
        ball.x -= ball.left
        bvx = -bvx
        bvy = 192 * np.sign(bvy)
        
        sr += 1
        scorer.text = str(sr)
        
    if bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            app.play_once(sound)
            ball.x -= ball.right - padr.left
            bvx = -bvx
            bvy += pvy / 2
            
    if bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            app.play_once(sound)
            ball.x += ball.left - padl.right
            bvx = -bvx
            bvy += pvy / 2
            
    ball.wrap_position(WIDTH, HEIGHT)

In [21]:
app.run()

Canvas(layout=Layout(height='512px', width='512px'), size=(512, 512))